# **Federated learning server without local data sharing**

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd

import json

import sys
sys.path.append("../")
import neural_network
import utils

import infer_tools
import participant

import concurrent.futures
#from multiprocessing.pool import ThreadPool

from tqdm.notebook import trange

2023-01-30 17:41:12.950609: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 17:41:12.953783: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 17:41:12.953917: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    except RuntimeError as e:
        print(e)

## **Preparations**

In [3]:
with open("../../02_data/filtered_vehicles.json", "r") as f:
    filtered_vehicles = json.load(f)
with open("../../dockeroutput/day_0/vehicle_maps.json", "r") as f:
    vehicle_maps = json.load(f)
    veh_to_idx_map = vehicle_maps["vehicle_to_idx_map"]
    idx_to_veh_map = vehicle_maps["idx_to_vehicle_map"]

In [4]:
filtered_vehicles = filtered_vehicles["vehicles"]

In [5]:
settings = {
    "min": 18220,
    "max": 46800,
    "mean": 0.59,
    "std": 0.35302
}

PARKING_ID_LIST = [i for i in range(1059, 1186+1)]

In [6]:
test_t = np.arange(14460, 50400+1, 60)
test_t = utils.normalize(test_t, settings["min"], settings["max"])

In [7]:
oracle = pd.read_csv("../../02_data/oracle.csv")

In [8]:
plist=[i for i in range(1059,1186+1)]
plist.remove(1148)
parking_testset = None
for parking in plist:
    id_encoding = utils.one_hot_encoder([parking]*len(test_t), PARKING_ID_LIST)
    test_d = np.hstack([id_encoding, test_t.reshape(len(test_t), 1)])
    if parking_testset is None:
        parking_testset = test_d
    else:
        parking_testset = np.vstack([parking_testset, test_d])

In [9]:
with open("../../02_data/known_parkings.json", "r") as f:
    known_parkings_by_vehicles = json.load(f)
with open("../../02_data/known_moving_times.json", "r") as f:
    known_moving_times = json.load(f)

## **Learning**

In [10]:
NUM_VEHICLES = 50

In [11]:
participants = [participant.FLParticipant() for i in range(NUM_VEHICLES)]

2023-01-30 17:41:14.834949: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-30 17:41:14.835469: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 17:41:14.835629: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 17:41:14.835736: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [12]:
def train_participant(parameters):
    def _train(participant, weights, veh_num, day):
        return participant.train(weights, veh_num, day, epochs=50)

    def _eval(participant, veh_id, baseline_model):
        true_parkings = known_parkings_by_vehicles[veh_id]
        true_time = known_moving_times[veh_id]
        return infer_tools.evaluate_performance(participant.nn.model, oracle, baseline_model, parking_testset,
                                               test_t, settings, true_parkings, true_time)
    
    partip = participants[parameters[0]]
    veh_num = veh_to_idx_map[parameters[2]]
    baseline = parameters[4]
    weights, samp_num = _train(partip, parameters[1], veh_num, parameters[3])
    results = _eval(partip, parameters[2], baseline)
    
    return weights, samp_num, results[0], results[1]

In [13]:
def fed_avg(parameters, weights):
        new_params = []
        for layer_i in range(len(parameters[0])):
            
            new_params.append(
                np.average([parameters[i][layer_i] for i in range(len(parameters))],
                           weights=weights,
                           axis=0
                )
            )
        return new_params

In [14]:
fed_model = neural_network.NeuralNetwork()

fed_weights = fed_model.model.get_weights()

pool = concurrent.futures.ThreadPoolExecutor(max_workers=NUM_VEHICLES)
#pool = ThreadPool(NUM_VEHICLES)

position_accuracies = []
time_accuracies = []


for day in trange(55):
    selected_participants = np.random.choice(np.arange(0, len(filtered_vehicles)), NUM_VEHICLES, replace=False)
    selected_vehicles = [filtered_vehicles[i] for i in selected_participants]
    
    params = []
    for i in range(len(selected_vehicles)):
        params.append([i, fed_weights, selected_vehicles[i], day, fed_model.model])
    
    results = pool.map(train_participant, params)
    weights = []
    num_samps = []
    p_acc, t_acc = [], []
    for r in results:
        weights.append(r[0])
        num_samps.append(r[1])
        p_acc.append(r[2])
        t_acc.append(r[3])
    position_accuracies.append(p_acc)
    time_accuracies.append(t_acc)
    fed_model.model.set_weights(fed_weights)
    
    fed_weights = fed_avg(weights, num_samps)

  0%|          | 0/55 [00:00<?, ?it/s]

2023-01-30 17:41:41.312380: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [15]:
save = {
    "position_accuracies": position_accuracies,
    "time_accuracies": time_accuracies
}

In [16]:
with open("../../02_data/fed_accuracies.json", "w") as f:
    json.dump(save, f)

In [17]:
fed_model.model.save("../../04_nn_models/fed_noshare")

INFO:tensorflow:Assets written to: ../../04_nn_models/fed_noshare/assets
